In [1]:
import polars as pl
import psycopg
from psycopg import sql
import spacy
from uuid import UUID
from spacy.tokens import Doc, Token, Span
from spacy.language import Language
from numerizer import numerize


In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
all_lyrics =pl.scan_csv("song_lyrics.csv")

all_lyrics.select('tag').unique().collect()

tag
str
"""misc"""
"""rap"""
"""country"""
"""pop"""
"""rock"""
"""rb"""


In [4]:
from polars import DataFrame



In [31]:
# first  = all_lyrics.filter(pl.col("tag").is_in(["rap",  "rb", "other"])).filter(pl.col('lyrics').str.contains('fi̇ve')).collect()
first  = all_lyrics.filter(pl.col('lyrics').str.contains('fi̇ve')).collect()


In [35]:
song = all_lyrics.filter(pl.col('title')=="OPERASYON").collect()

In [36]:
song

title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
str,str,str,i64,i64,str,str,i64,str,str,str
"""OPERASYON""","""rap""","""FFG""",2021,150,"""{Rek,Mute}""","""Verse 1 (Nus) OPERASYON VAR F…",6818494,"""tr""",null,null


In [32]:
first
# print(first['lyrics'][0])


title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
str,str,str,i64,i64,str,str,i64,str,str,str


In [5]:
rap_q = pl.scan_csv("song_lyrics.csv").filter(pl.col("tag").is_in(["rap", "misc", "rb"]))

rap = rap_q.collect()


In [13]:
print(rap[1532045]['lyrics'].item())

Hook:
I'm an anomaly rap name lobotomy
I'ma lost species I stand alone
If I run it up they gonna hate
Money conversation fırst to debate
Kept it gangsta in and outta the streets
Big bag cookies smoking the trees
Dabbing and slabbing im never on E

I'm an anomaly rap name lobotomy
I'ma lost species I stand alone
If I run it up they gonna hate
Money conversation fırst to debate
Kept it gangsta in and outta the streets
Big bag cookies smoking the trees
Dabbing and slabbing im never on E

Verse:
That couldn't be me I move so different
Wait my turn sit back and I listen
Real to some a jokе to others
Keep the essеntials that is a bundle
It is a legion of raps stashed in my head call it a maze
No l'm not emo wear black every day
I'm from the L but my bro from the k
Run up the cheese we trapping those rats
Beat up the beat like I was the bat
Run up a check feel like l'm Bruce
How many blunts you smoke (Bout 2)
HD greet while yall smoking foo
I couldn't be you not in the least
I pray and i pray

In [23]:
print(rap.shape)

(2102733, 11)


In [ ]:
rap_q = pl.scan_csv("song_lyrics.csv").filter(pl.col("tag").is_in(["rap", "misc", "rb"])).limit(10)

rap = rap_q.collect()
rap.shape

(10, 11)

In [26]:
rap.write_csv("./tests/test_data.csv")

In [18]:
# class Ngram:
#     """A cleaned ngram from a song.

#     We store the clean ngram text, but the indecies where the ngram occurs in the raw
#     lyrics. This halves the required storage while allowing us to pull the original
#     lyrics from the song. Because we remove stopwords, len(clean_text) may not equal
#     len(text).
#     """

#     clean_text: str
#     start_idx_unprocessed: int
#     end_idx_unprocessed: int


def is_valid_token(token: Token) -> bool:
    # Is stop includes numbers. Go over this an decide if it's desired
    token_is_invalid = token.is_stop or token.is_punct or token.is_space
    return not token_is_invalid


def get_ngram_component_from_token(token: Token) -> str:
    """Determine the appropriate string representation of a token."""
    if numerized := token._.numerized != token.text:
        return numerized
    return token.lemma_


def ngram_generator(lyrics: Doc, ngram_length=5) -> list[Token]:
    """Iterate through a song, yielding lists of ngrams until no full ones remain."""

    tokens_in_ngram: list[Token] = []
    for potential_token in lyrics:
        if len(tokens_in_ngram) < ngram_length:
            if is_valid_token(potential_token):
                tokens_in_ngram.append(potential_token)

        # to_yield = Ngram(
        #     clean_text=" ".join(token.lemma_ for token in tokens_in_ngram),
        #     start_idx_unprocessed=tokens_in_ngram[0].idx,
        #     end_idx_unprocessed=tokens_in_ngram[-1].idx+len(tokens_in_ngram[-1].idx),
        # )
        elif len(tokens_in_ngram) < ngram_length:
            raise StopIteration
        else:
            tokens_to_return = tokens_in_ngram
            tokens_in_ngram = []
            yield tokens_to_return

In [21]:
from spacy.tokens.token import Token


def generate_ngrams_from_lyrics(lyrics: str, song_id: UUID) -> list[Token]:
    doc: Doc = nlp(lyrics)

    # Here we should transform these lists of tokens into Ngram objects that will
    # match up with the DB table
    return [ngram for ngram in ngram_generator(doc)]


In [ ]:
def _token_to_db(ngram: list[Token], song_id: UUID) -> tuple[str, str, int, int]:
    ngram_text = " ".join([token.text for token in ngram])
    start_index = ngram[0].idx
    end_index = ngram[-1] + len(ngram[-1])

    return (ngram_text, song_id, start_index, end_index)


def insert_ngrams(lyrics: str, song_id: UUID):
    """UNTESTED, also need to inject a connection (or restructure or something)"""
    ngrams = generate_ngrams_from_lyrics(lyrics, song_id)

    query = sql.SQL(
        "INSERT INTO ngrams (ngram, song_id, start_in_song, end_in_song) VALUES (%s, %s, %s, %s);"
    )
    res = cur.executemany(
        query, [_token_to_db(ngram, song_id) for ngram in ngrams]
    )

    conn.commit()
    return res.fetchone()[0]

In [24]:
for song in rap:

res = generate_ngrams_from_lyrics(rap[1]['lyrics'].item(), 1)

print(res)

KeyboardInterrupt: 

In [3]:
def generate_song_insert_values(song:dict[str, str]) ->  tuple[str]:
    lyrics = song['lyrics']


    values = (song['artist'], lyrics)
    return  values


In [ ]:
generate_song_insert_values(next(rap[1].iter_rows(named=True)))

In [29]:


def insert_song(values:tuple[str], conn, cur)-> UUID:
        query = sql.SQL(
            "INSERT INTO songs (artist, lyrics) VALUES (%s, %s) RETURNING id;"
        )
        res = cur.execute(query, values)

        conn.commit()
        return res.fetchone()[0]

In [ ]:
with psycopg.connect("host=localhost user=postgres password=pass1234") as conn:
    with conn.cursor() as cur:
        ctx = {'conn': conn, 'cur': cur}

        for i, song in enumerate(rap.iter_rows(named=True)):
            if i >= 5:
                break
            song_values = generate_song_insert_values(song)

            song_id = insert_song(song_values, **ctx)
            insert_ngrams(song['lyrics'], song_id)

            print([c for c in cur])

In [58]:
l=[1]
l.append([12])
l

t = ()
t+=((2,),)
t+=((3,),)
t

((2,), (3,))